In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [2]:
os.listdir('Data')

['Clean_test_data.csv',
 'Clean_train_data.csv',
 'holidays.xlsx',
 'processed_test.csv',
 'processed_train.csv',
 'promotional_expense.csv',
 'sample_submission.csv',
 'test_visual.csv',
 'train_visual.csv',
 'yds_test2018.csv',
 'yds_train2018.csv']

In [3]:
train_data = pd.read_csv('Data/yds_train2018.csv')
test_data = pd.read_csv('Data/yds_test2018.csv')

promotional_data = pd.read_csv('Data/promotional_expense.csv')
holiday_data = pd.read_excel('Data/holidays.xlsx')

In [4]:
train_data.shape,test_data.shape,promotional_data.shape,holiday_data.shape

((79072, 8), (105, 6), (435, 5), (652, 3))

In [5]:
train_data.head()

,S_No,Year,Month,Week,Merchant_ID,Product_ID,Country,Sales
0,1,2013,1,1,ar00001,1,Argentina,157500.0
1,2,2013,1,1,ar00003,1,Argentina,39375.0
2,3,2013,1,1,ar00004,1,Argentina,15750.0
3,4,2013,1,1,ar00007,1,Argentina,47250.0
4,5,2013,1,1,ar00008,1,Argentina,283500.0


In [6]:
test_data.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,NaN
1,79074,2016,5,1,Argentina,NaN
2,79075,2016,6,1,Argentina,NaN
3,79076,2016,7,1,Argentina,NaN
4,79077,2016,8,1,Argentina,NaN


In [7]:
holiday_data.head()

,Date,Country,Holiday
0,"2014, 1, 1",Denmark,Nytårsdag
1,"2014, 4, 17",Denmark,Skærtorsdag
2,"2014, 4, 18",Denmark,Langfredag
3,"2014, 4, 20",Denmark,Påskedag
4,"2014, 4, 21",Denmark,Anden påskedag


In [8]:
promotional_data.head()

,Year,Month,Country,Product_Type,Expense_Price
0,2013,1,Argentina,1,14749.307
1,2013,1,Argentina,2,1329.374
2,2013,1,Belgium,2,249.590
3,2013,1,Columbia,1,1893.122
4,2013,1,Columbia,2,1436.726


In [9]:
promotional_data.rename(columns={'Product_Type':'Product_ID'},inplace=True)

<h3>Extracting month,year and day from holiday_data date column</h3>

In [10]:
def to_date(x=None):
    return pd.to_datetime('-'.join(x.split(',')))

holiday_data['Date'] = holiday_data['Date'].apply(lambda y : to_date(x=y))

In [11]:
holiday_data['Month'] = holiday_data['Date'].dt.month
holiday_data['Year'] = holiday_data['Date'].dt.year

In [12]:
holiday_data.head()

,Date,Country,Holiday,Month,Year
0,2014-01-01,Denmark,Nytårsdag,1,2014
1,2014-04-17,Denmark,Skærtorsdag,4,2014
2,2014-04-18,Denmark,Langfredag,4,2014
3,2014-04-20,Denmark,Påskedag,4,2014
4,2014-04-21,Denmark,Anden påskedag,4,2014


In [40]:
train_data = train_data.merge(right=holiday_data[[i for i in holiday_data.columns if i not in ['Date']]],on=['Year','Month','Country'],how='left')
train_data = train_data.merge(right=promotional_data,on=['Year','Month','Country','Product_ID'],how='left')

In [41]:
test_data = test_data.merge(right=holiday_data[[i for i in holiday_data.columns if i not in ['Date']]],on=['Year','Month','Country'],how='left')
test_data = test_data.merge(right=promotional_data,on=['Year','Month','Country','Product_ID'],how='left')

In [42]:
test_data.head()

,S_No,Year,Month,Product_ID,Country,Sales,Holiday,Expense_Price
0,79073,2016,4,1,Argentina,NaN,Día del Veterano y de los Caidos en la Guerra...,8214.875
1,79074,2016,5,1,Argentina,NaN,Día del Trabajo [Labour Day],10777.878
2,79074,2016,5,1,Argentina,NaN,Día de la Revolucion de Mayo [May Revolution ...,10777.878
3,79075,2016,6,1,Argentina,NaN,Día Pase a la Inmortalidad del General Martín...,10320.673
4,79075,2016,6,1,Argentina,NaN,Día Pase a la Inmortalidad del General D. Man...,10320.673


In [43]:
train_data.head()

,S_No,Year,Month,Week,Merchant_ID,Product_ID,Country,Sales,Holiday,Expense_Price
0,1,2013,1,1,ar00001,1,Argentina,157500.0,"""Año Nuevo [New Years Day]""",14749.307
1,2,2013,1,1,ar00003,1,Argentina,39375.0,"""Año Nuevo [New Years Day]""",14749.307
2,3,2013,1,1,ar00004,1,Argentina,15750.0,"""Año Nuevo [New Years Day]""",14749.307
3,4,2013,1,1,ar00007,1,Argentina,47250.0,"""Año Nuevo [New Years Day]""",14749.307
4,5,2013,1,1,ar00008,1,Argentina,283500.0,"""Año Nuevo [New Years Day]""",14749.307


In [44]:
# converting train weekly data to monthly
train_data = train_data.groupby(['Year','Month','Country','Product_ID'],as_index=False).agg({'Sales':'sum','Holiday':'nunique','Expense_Price':'mean'})
test_data = test_data.groupby(['Year','Month','Country','Product_ID'],as_index=False).agg({'Holiday':'nunique','Expense_Price':'mean'})


In [45]:
train_data = train_data.sort_values(['Year','Month','Country','Product_ID'])
test_data = test_data.sort_values(['Year','Month','Country','Product_ID'])

In [46]:
# train_data['expense_sales_ratio'] = train_data['Expense_Price']/train_data['Sales']
# test_data['expense_sales_ratio'] = test_data['Expense_Price']/test_data['Sales']

In [47]:
train_data.head()

,Year,Month,Country,Product_ID,Sales,Holiday,Expense_Price
0,2013,1,Argentina,1,34346025.00,1,14749.307
1,2013,1,Argentina,2,2751851.48,1,1329.374
2,2013,1,Belgium,2,314850.13,1,249.590
3,2013,1,Columbia,1,7894712.62,2,1893.122
4,2013,1,Columbia,2,5770274.12,2,1436.726


In [48]:
test_data.head()

,Year,Month,Country,Product_ID,Holiday,Expense_Price
0,2015,11,England,4,0,171.166
1,2015,11,England,5,0,416.969
2,2015,11,Finland,4,0,1306.731
3,2015,12,England,4,3,194.468
4,2015,12,England,5,3,464.351


In [49]:
train_data.shape,test_data.shape

((388, 7), (105, 6))

In [50]:
old_test = pd.read_csv('Data/yds_test2018.csv')

In [51]:
old_test.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,NaN
1,79074,2016,5,1,Argentina,NaN
2,79075,2016,6,1,Argentina,NaN
3,79076,2016,7,1,Argentina,NaN
4,79077,2016,8,1,Argentina,NaN


In [52]:
test_data = test_data.merge(right=old_test.drop(['Sales'],1),on=['Year','Month','Product_ID','Country'],how='left')

In [53]:
test_data.head()

,Year,Month,Country,Product_ID,Holiday,Expense_Price,S_No
0,2015,11,England,4,0,171.166,79160
1,2015,11,England,5,0,416.969,79151
2,2015,11,Finland,4,0,1306.731,79169
3,2015,12,England,4,3,194.468,79161
4,2015,12,England,5,3,464.351,79152


In [54]:
len(set(test_data['S_No'])) == test_data.shape[0]

True

In [55]:
train_data.to_csv('Data/Clean_train_data.csv',index=False)
test_data.to_csv('Data/Clean_test_data.csv',index=False)